# SRCMEngine HybridModel — quickstart patterns

This is a compact reference notebook showing the *standard workflow* for the user API:

1. Create a model with `HybridModel(species=[...])`
2. Configure `domain`, `diffusion`, `conversion`
3. Provide PDE reaction terms via `reaction_terms(lambda ...: (...))`
4. Add macroscopic reactions with `add_reaction(...)` (auto-decomposition)
5. `build(rates=...)`
6. Provide initial conditions as NumPy arrays and run

It also includes a couple of “gotchas” and sanity checks.


In [3]:
import sys
from pathlib import Path

repo_root = Path.cwd().parent   # examples/ → repo/
sys.path.append(str(repo_root))

In [5]:
import numpy as np
from srcm_engine import HybridModel


## Minimal skeleton

In [6]:
# Replace species and rates to suit your model
m = HybridModel(species=["X", "Y"])

# 1) domain
m.domain(L=1.0, K=10, pde_multiple=4, boundary="zero-flux")

# 2) diffusion (must supply all species)
m.diffusion(X=0.01, Y=0.02)

# 3) conversion
m.conversion(threshold=50, rate=1.0)

# 4) PDE reaction terms: returns (dX, dY)
m.reaction_terms(lambda X, Y, r: (
    -r["k"] * X,
    +r["k"] * X,
))

# 5) macroscopic reactions -> hybrid decomposition
m.add_reaction({"X": 1}, {"Y": 1}, rate_name="k")

# 6) build with numeric rates
m.build(rates={"k": 0.1})

m.describe_reactions()


0.0

 Reaction system description

=== Macroscopic reactions ===
[1] X → Y    (rate_name='k')    (rate=0.1)
    Decomposed hybrid reactions:
      - [1] k_D
           Reactants: {'D_X': 1}
           Products : {'D_Y': 1}
           State Δ  : {'D_X': -1, 'D_Y': 1}
           Info     : D_X → D_Y


## Initial condition shapes

In [7]:
K = m.domain_obj.K
n_pde = m.domain_obj.n_pde
n_species = len(m.species)

init_ssa = np.zeros((n_species, K), dtype=int)
init_pde = np.zeros((n_species, n_pde), dtype=float)

# Example: put 20 particles of X in compartment 0
init_ssa[0, 0] = 20

init_ssa.shape, init_pde.shape


((2, 10), (2, 40))

## Run a single simulation

In [8]:
res = m.run(init_ssa, init_pde, time=1.0, dt=0.01, seed=0)
res.ssa.shape, res.pde.shape


((2, 10, 101), (2, 40, 101))

## Gotchas / checks

- **All species must appear** in `diffusion(...)`.
- `reaction_terms(...)` must return **one array per species** (or a stacked array of shape `(n_species, Npde)`).
- Initial conditions must have shapes:
  - `init_ssa`: `(n_species, K)` integer
  - `init_pde`: `(n_species, n_pde)` float
- If a reaction rate is missing from `build(rates=...)`, propensities will KeyError when the engine runs.
